In [ ]:
import numpy as np 
import pandas as pd 
import scipy.stats as stats
import sys,os,glob
from functools import partial
import cv2

import torchvision
from torchvision import datasets, transforms

import torch
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
import torch.nn.functional as F
import torch.nn.init as init
import torch.nn as nn
from sys import float_info


import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,MultiplicativeNoise,ColorJitter,
    Transpose, ShiftScaleRotate, Blur,GaussianBlur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise,GaussNoise,IAAAffine, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomSizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)
from albumentations.pytorch import ToTensorV2

In [ ]:
FLAGS = {}
FLAGS['batch_size'] = 64
FLAGS['num_workers'] = 4
FLAGS['learning_rate'] = 0.01
FLAGS['num_cores'] = 1
FLAGS['num_epochs'] = 2
FLAGS['img_size'] = (256,256)

ROOT = "../input/cassava-leaf-disease-merged/"
trainpath = ROOT + 'train/'
df = pd.read_csv("../input/cassava-leaf-disease-merged/merged.csv")
df.head()

In [ ]:
import torchvision.models as models
class Model(nn.Module):
    def __init__(self, ovnum = 10):
        super(Model, self).__init__()
        self.vgg = torchvision.models.vgg16_bn(pretrained=False).features[0:17]
        weight = self.vgg[0].weight
        self.vgg[0] = nn.Conv2d(5, 64, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1), bias=False)
        self.vgg[0].weight = torch.nn.Parameter(torch.cat([weight,weight[:,:2,:,:]],dim=1))
        #self.mid = nn.Sequential(nn.Conv2d(256,512, 3, 1, 1,bias=False),nn.BatchNorm2d(512),nn.ReLU())
        self.out =   nn.Sequential(nn.Conv2d(256,5, 1, 1, 0,bias=False),nn.Softmax2d())
        self.ovout = nn.Sequential(nn.Conv2d(256,ovnum, 1, 1, 0,bias=False),nn.Softmax2d())
    def forward(self, x):
        x = self.vgg(x)
        #x = self.mid(x)
        x_out = self.out(x)
        x_out_oc = self.ovout(x)
        y = F.interpolate(x_out, size=FLAGS['img_size'], mode="bilinear", align_corners=False)
        y_ov = F.interpolate(x_out_oc, size=FLAGS['img_size'], mode="bilinear", align_corners=False)
        return y, y_ov

In [ ]:
model = Model(ovnum= 10)
model.load_state_dict(torch.load('../input/training-iic-segmentations-cassava/model_epoch0.pt'))

In [ ]:
def get_train_transforms():
    return Compose([            
            #Transpose(p=0.5),
            #HorizontalFlip(p=0.5),
            #VerticalFlip(p=0.5),
            #RandomSizedCrop(min_max_height=((16, 24)),height=28,width=28,p=0.20),
            #MultiplicativeNoise(p=1.0,multiplier=(0.8,1.2),elementwise=False),
            #GaussianBlur(p=1.0,blur_limit=(3,7)),
            #Cutout(num_holes=8, max_h_size=8, max_w_size=8, p=1.0),
            ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.125,p=1.0),
        ], p=1.)
          
def tensor_transform():
    return Compose([ToTensorV2()], p=1.)

def random_affine(img,device,min_rot=None, max_rot=None, min_shear=None,max_shear=None, min_scale=None, max_scale=None):
    assert (len(img.shape) == 3)
    a = np.radians(np.random.rand() * (max_rot - min_rot) + min_rot)
    shear = np.radians(np.random.rand() * (max_shear - min_shear) + min_shear)
    scale = np.random.rand() * (max_scale - min_scale) + min_scale
    affine1_to_2 = np.array([[np.cos(a) * scale, - np.sin(a + shear) * scale, 0.],
                           [np.sin(a) * scale, np.cos(a + shear) * scale, 0.],
                           [0., 0., 1.]], dtype=np.float32)  # 3x3

    affine2_to_1 = np.linalg.inv(affine1_to_2).astype(np.float32)

    affine1_to_2, affine2_to_1 = affine1_to_2[:2, :], affine2_to_1[:2, :]  # 2x3
    affine1_to_2, affine2_to_1 = torch.from_numpy(affine1_to_2).to(device), torch.from_numpy(affine2_to_1).to(device)

    img = perform_affine_tf(img.unsqueeze(dim=0), affine1_to_2.unsqueeze(dim=0))
    img = img.squeeze(dim=0)

    return img, affine1_to_2, affine2_to_1

def perform_affine_tf(data, tf_matrices):
  # expects 4D tensor, we preserve gradients if there are any

    n_i, k, h, w = data.shape
    n_i2, r, c = tf_matrices.shape
    assert (n_i == n_i2)
    assert (r == 2 and c == 3)

    grid = F.affine_grid(tf_matrices, data.shape)  # output should be same size
    data_tf = F.grid_sample(data, grid,
                          padding_mode="zeros")  # this can ONLY do bilinear

    return data_tf

In [ ]:
class torchdatasets_forIIC(Dataset):
    def __init__(self,df,transform,noisetransform,device='cpu'):
        self.df = df
        self.transform = transform
        self.noisetransform = noisetransform
        self.device = device

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path_list = self.df.image_id.values
        path = trainpath + path_list[idx]
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, FLAGS['img_size'])
        #image = tfcrop()(image=image)["image"]
        assert(image.shape == (*FLAGS['img_size'],3))
        org_image=np.array(image, dtype='float32')/255.0        
        proc_image=np.array(image,dtype=np.float32)

        #change orignal image to sobel image 
        img_gray = cv2.cvtColor(proc_image, cv2.COLOR_RGB2GRAY)
        dx = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=3)
        dy = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=3)
        xmax = np.max(abs(dx))
        ymax = np.max(abs(dy))
        sobel_img_x = np.float32(dx[:,:,None]/xmax)
        sobel_img_y = np.float32(dy[:,:,None]/ymax)
        data_target = np.concatenate([org_image,sobel_img_x,sobel_img_y],axis=2)
        data_target = self.transform(image=data_target)["image"]
        
        #change augment image to sobel image 
        augimage = self.noisetransform(image=proc_image)["image"]
        aug_gray = cv2.cvtColor(augimage, cv2.COLOR_RGB2GRAY)
        dx_tf = cv2.Sobel(aug_gray, cv2.CV_64F, 1, 0, ksize=3)
        dy_tf = cv2.Sobel(aug_gray, cv2.CV_64F, 0, 1, ksize=3) 
        sobel_img_x_tf = np.float32(dx_tf[:,:,None]/xmax)
        sobel_img_y_tf = np.float32(dy_tf[:,:,None]/ymax)
        org_image_tf = self.noisetransform(image=org_image)["image"]
        data_other = np.concatenate([org_image_tf,sobel_img_x_tf,sobel_img_y_tf],axis=2)
        data_other = self.transform(image=data_other)["image"]
        data_other,affine1_to_2,affine2_to_1=random_affine(data_other, min_rot=-30, max_rot=30, min_shear=-10,
                                                         max_shear=10, min_scale=0.8, max_scale=1.2, device=self.device)
        return affine2_to_1, data_target, data_other

In [ ]:
noisetransform = get_train_transforms()
transform = tensor_transform()
test_ds = torchdatasets_forIIC(df=df,transform=transform,noisetransform=noisetransform,device='cpu')
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=4,
                          shuffle=False, num_workers=4,drop_last=False)

In [ ]:
col = 4
row = 3
plt.figure(figsize=(col*5,row*5/0.90))
num = 0 
model.eval()

for affine2_to_1,data,gdata in test_loader:
    if num==col*row:
        break
    #print(data.shape,gdata.shape)
    A =model(data)[0].detach().numpy()
    for j in range(4):
        if num==col*row:
            break
        image = A[j]
        newimg = np.zeros(image[0][0].shape)
        for i in range(5):
            piece = image[i]
            piece[piece>0.95]=i
            newimg = newimg + piece
        num += 1
        plt.subplot(row, col, num)
        plt.imshow(data[j][:3].permute(1,2,0))
        #plt.show()
        num += 1
        plt.subplot(row, col, num)
        plt.imshow(newimg)#.transpose(1,2,0))
        #plt.show()
plt.show()